In [1]:
import os

import numpy as np
import pandas as pd

from tensorflow import keras

In [2]:
train_raw = pd.read_csv('data/train.csv').drop('id', axis=1).to_numpy()
test_raw = pd.read_csv('data/test.csv', dtype=float)
split = int(train_raw.shape[0] * .6)
X_train = train_raw[:split, :14]
y_train = train_raw[:split, 14:]
X_valid = train_raw[split:, :14]
y_valid = train_raw[split:, 14:]
X_valid, X_test = np.array_split(X_valid, 2)
y_valid, y_test = np.array_split(y_valid, 2)

In [3]:
# sample size, time steps, input dimension
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_valid = X_valid.reshape(X_valid.shape[0], X_valid.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [4]:
conv_model = keras.models.Sequential([
    keras.layers.Conv1D(128, kernel_size=7, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    keras.layers.Conv1D(256, kernel_size=3, activation='relu'),
    keras.layers.MaxPool1D(pool_size=3),
    keras.layers.Flatten(),
    keras.layers.Dense(48, activation='elu'),
    keras.layers.Dense(1, activation='elu')
])
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                               min_delta=1e-4)

conv_model.compile(optimizer='adam', loss='mse')
conv_model.fit(X_train, y_train, epochs=60, batch_size=128,
               callbacks=[early_stopping],
               validation_data=(X_valid, y_valid))

Epoch 1/60
1407/1407 [==============================] - 9s 4ms/step - loss: 0.9827 - val_loss: 0.5317
Epoch 2/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5359 - val_loss: 0.5249
Epoch 3/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5344 - val_loss: 0.5251
Epoch 4/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5322 - val_loss: 0.5205
Epoch 5/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5279 - val_loss: 0.5305
Epoch 6/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5277 - val_loss: 0.5160
Epoch 7/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5247 - val_loss: 0.5148
Epoch 8/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5219 - val_loss: 0.5171
Epoch 9/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.5214 - val_loss: 0.5218
Epoch 10/60
1407/1407 [==============================] - 6s 4ms/step - loss: 0.519

In [5]:
conv_model.evaluate(X_test, y_test)

1875/1875 [==============================] - 3s 2ms/step - loss: 0.5070


0.5069503784179688

In [7]:
import xgboost as xgb


split = int(train_raw.shape[0] * .7)
X_train_xgb = train_raw[:split, :14]
y_train_xgb = train_raw[:split, 14:]


regressor = xgb.XGBRegressor(
    colsample_bytree=0.7446950226284034,
    gamma=0.697776868966826,
    learning_rate=0.11205606010503663,
    max_bin=538,
    max_depth=9,
    min_child_weight=2.1057925465153993)

regressor.fit(X_train_xgb, y_train_xgb)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.7446950226284034, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.697776868966826, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.11205606010503663, max_bin=538,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=9, max_leaves=None,
             min_child_weight=2.1057925465153993, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, random_state=None, ...)

In [15]:
rnn_model = keras.models.Sequential([
    keras.layers.GRU(30, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    keras.layers.GRU(30),
    keras.layers.Dropout(.3),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(1, activation='elu')
])

rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit(X_train, y_train, epochs=200, batch_size=128,
              callbacks=[early_stopping],
              validation_data=(X_valid, y_valid))

Epoch 1/200
1407/1407 [==============================] - 15s 9ms/step - loss: 1.9210 - val_loss: 0.5535
Epoch 2/200
1407/1407 [==============================] - 12s 8ms/step - loss: 0.5540 - val_loss: 0.6002
Epoch 3/200
1407/1407 [==============================] - 12s 8ms/step - loss: 0.5455 - val_loss: 0.5358
Epoch 4/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.5437 - val_loss: 0.5391
Epoch 5/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.5420 - val_loss: 0.5332
Epoch 6/200
1407/1407 [==============================] - 12s 8ms/step - loss: 0.5403 - val_loss: 0.5317
Epoch 7/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.5384 - val_loss: 0.5313
Epoch 8/200
1407/1407 [==============================] - 12s 9ms/step - loss: 0.5359 - val_loss: 0.5273
Epoch 9/200
1407/1407 [==============================] - 11s 8ms/step - loss: 0.5337 - val_loss: 0.5252
Epoch 10/200
1407/1407 [==============================] - 12s 8m

In [11]:
rnn_model.evaluate(X_test, y_test)

1875/1875 [==============================] - 7s 4ms/step - loss: 0.5058


INFO:tensorflow:Assets written to: saved_models\rnn_baseline\assets


INFO:tensorflow:Assets written to: saved_models\rnn_baseline\assets


In [12]:
def predict_ensemble(test_raw: np.array) -> np.array:
    nn_test = test_raw.reshape(test_raw.shape[0], test_raw.shape[1], 1)

    conv_predictions = conv_model.predict(nn_test)
    rnn_predictions = rnn_model.predict(nn_test)
    xgb_predictions = regressor.predict(test_raw)
    stacked_predictions = np.column_stack((conv_predictions, rnn_predictions, xgb_predictions))
    target = np.mean(stacked_predictions, axis=1)

    return target

In [40]:
def prepare_for_rnn(data, n_in=1, n_out=1):
    n_vars = data.shape[1]
    df = pd.DataFrame(data)
    cols, names = [], []

    for i in range(n_in, 0, -1):
        shifted = df.shift(i).fillna(0)

        cols.append(shifted)

        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

    for i in range(0, n_out):
        shifted = df.shift(-i).fillna(0)

        cols.append(shifted)

        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names

    return agg


train_rnn = prepare_for_rnn(train_raw, 1, 20)
values = train_rnn.values
train_rnn = values[:split, :]
valid_rnn = values[split:, :]
X_train_rnn, y_train_rnn = train_rnn[:, :-1], train_rnn[:, -1]
X_valid_rnn, y_valid_rnn = valid_rnn[:, :-1], valid_rnn[:, -1]
X_train_rnn = X_train_rnn.reshape((X_train_rnn.shape[0], 1, X_train_rnn.shape[1]))
X_valid_rnn = X_valid_rnn.reshape((X_valid_rnn.shape[0], 1, X_valid_rnn.shape[1]))

In [41]:
rnn_model_diff_data = keras.models.Sequential([
    keras.layers.GRU(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    keras.layers.Dropout(.3),
    keras.layers.GRU(100),
    keras.layers.Dropout(.3),
    keras.layers.Dense(32, activation='elu'),
    keras.layers.Dense(1, activation='elu')
])
early_stopping_rnn = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20,
                                                   min_delta=1e-4)

rnn_model_diff_data.compile(optimizer='adam', loss='mse')
rnn_model_diff_data.fit(X_train, y_train, epochs=200, batch_size=128,
              callbacks=[early_stopping_rnn],
              validation_data=(X_valid, y_valid))

Epoch 1/200
1407/1407 [==============================] - 17s 11ms/step - loss: 1.1432 - val_loss: 0.5567
Epoch 2/200
1407/1407 [==============================] - 14s 10ms/step - loss: 0.5690 - val_loss: 0.5513
Epoch 3/200
1407/1407 [==============================] - 14s 10ms/step - loss: 0.5496 - val_loss: 0.5630
Epoch 4/200
1407/1407 [==============================] - 14s 10ms/step - loss: 0.5433 - val_loss: 0.5328
Epoch 5/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.5404 - val_loss: 0.5304
Epoch 6/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.5371 - val_loss: 0.5291
Epoch 7/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.5339 - val_loss: 0.5262
Epoch 8/200
1407/1407 [==============================] - 13s 9ms/step - loss: 0.5314 - val_loss: 0.5265
Epoch 9/200
1407/1407 [==============================] - 14s 10ms/step - loss: 0.5298 - val_loss: 0.5273
Epoch 10/200
1407/1407 [==============================] - 1

In [14]:
y_target = predict_ensemble(test_raw.drop('id', axis=1).to_numpy())

pd\
    .DataFrame(np.column_stack([test_raw.loc[:, 'id'], y_target]), columns=['id', 'target'])\
    .astype({'id': int})\
    .to_csv(os.path.join('submissions', 'ensemble.csv'), index=False)

6250/6250 [==============================] - 16s 3ms/step
